# Если ОЗУ не хватает

In [ ]:
pip install fasttext

In [2]:

import fasttext
from huggingface_hub import hf_hub_download
import pandas as pd
import numpy as np
from tqdm import tqdm
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import csv
import sys

In [ ]:
from google.colab import drive # Подключение к диску с данными
drive.mount('/content/drive')

In [ ]:
model_path = hf_hub_download(repo_id="facebook/fasttext-ru-vectors", filename="model.bin") # Загрузка модели
model = fasttext.load_model(model_path)

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
def preprocess(text):
    """
    Обработка данных
    """
    text = list(filter(str.isalpha, word_tokenize(text.lower()))) # Токенизация
    text = list(lemmatizer.lemmatize(word) for word in text) # Лемматизация
    text = list(word for word in text if word not in stop_words) # Удаление стоп-слов
    return ' '.join(text)
stop_words = set(stopwords.words('english') + stopwords.words('russian') + ['xn'])
lemmatizer = WordNetLemmatizer()

In [ ]:
csv.field_size_limit(3000000000000000000) # Технические параметры

In [ ]:
df = pd.DataFrame() # Запись обработанного текста
lst = [] # Запись эмбеддингов
errors = [] # Запись ошибок

part = '4200_4500' # Часть названия файла

with open(f'/content/drive/MyDrive/Парсинг ИПС_1/parsing_{part}.csv', newline='') as f:
  reader = csv.reader(f)
  for row in tqdm(reader):
    try:
      clear_row = re.sub(r'\s+', ' ', ' '.join(row).replace('\n', ''))[2:]
      if clear_row == 'ERROR ERROR ERROR ERROR ERROR ERROR' or len(clear_row) == 0 or clear_row == 'url descr titl1 titl2 keyw text': # Отсечение строк, о которых не получить информацию
        continue
      row_final = preprocess(clear_row) # Обработка данных
      df = pd.concat((df, pd.DataFrame({'text': [row_final], 'url': [row[1]] }))) # Добавление текстовой информации ссылки в df
      lst.append(model[row_final]) # Добавление эмбеддинга текстовой информации в lst
    except:
      errors.append(row)

In [ ]:
len(errors)

In [12]:
df.to_csv(f'text_{part}.csv') # Сохранение текста
np.save(f'embed_{part}.npy', lst) # Сохранение эмбеддинга